## Emporio Analytics

## a) Business Context

Emporio Analytics collects granular transaction receipt data from independent single outlet/ single store grocery retailers. The collected data is parsed from printed paper receipts and is subjected to both data collection errors and store level nuances.

In this assessment, applicants are invited to suggest robust and scalable approaches to ensure data quality and develop analytical models to support monetization of insights. 

## b) Evaluation 

There are two different questions in the sections below. Applicants are expected to complete both questions in code (Python/R) or logic flows, and submitted in any presentation format (i.e HTML, PDF etc.) Applicants should summarise the steps that they undertook and describe their considerations taken. 


## c) Section 1: Quantity Estimation

### Dataset

Sales of two generic cigarettes across 488 stores were recorded for 2 weeks. Each cigarette is tracked through two IDs across independent stores, one for sale of a single-pack and one for sale of a multi-pack. 

#### Transaction Level Data 

<table align = "left">
    <tr>
        <th>product_ean</th>
        <th>store_id</th>
        <th>sales_receipt</th>
        <th>sales_pieces</th>
        <th>sales_turnover_gross</th>
    </tr>
    <tr>
        <th>Product ID</th>
        <th>Store ID</th>
        <th>Receipt ID</th>
        <th>No. Units transacted</th>
        <th>Total sales value,<br>
        denoted in Indonesian 
        Rupiah</th>
    </tr>
</table>

#### Product Master Data

<table align = "left">
    <tr>
        <th>product_ean</th>
        <th>product_name</th>
        <th>Is_multipack</th>
    </tr>
    <tr >
        <th>Product ID</th>
        <th>Generic Name of cigaratte</th>
        <th>Boolean flag to denote larger pack size</th>
    </tr>
</table>



### Requirement 

1. Investigate data quality problems within the transactional dataset and propose generalisable, scalable methods to manage identified isues.

In [162]:
# import relevant libraries for EDA 

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os 
import math

In [3]:
%matplotlib inline

In [10]:
data = {x.split('.csv')[0]:x for x in os.listdir() if '.csv' in x}

In [11]:
# def extract(filename):
#     return pd.
df1 = pd.DataFrame(pd.read_csv(data['s1_transaction_data']))

In [12]:
df1.head()

,product_ean,sales_pieces,sales_turnover_gross,store_id,sales_date_id,sales_receipt
0,9096004,3.0,65700.0,64,20191201,110342232
1,9096004,2.0,43800.0,64,20191205,111790500
2,9096004,1.0,21900.0,64,20191205,111839706
3,9096004,3.0,65700.0,64,20191204,111004662
4,9096004,1.0,21900.0,64,20191204,110997769


In [13]:
df2  = pd.DataFrame(pd.read_csv(data['s1_product_dimensions']))

In [14]:
print(len(df1),len(df1.drop_duplicates()), len(df1.dropna())) 
# there are 56658 rows in the entire datasetf or s1_transaction data
# there are 56500 rows when we drop duplicate rows, conclude that there are some rows that are duplicates in the data set
# there are 56658 rows when we drop rows that have na,that means all rows are filled


56658 56500 56658


To show that there are some records with more than 2 duplicates

In [15]:
df1[df1.duplicated()].drop_duplicates()

# some rows with more than 2 duplicates

,product_ean,sales_pieces,sales_turnover_gross,store_id,sales_date_id,sales_receipt
1021,9096004,1.0,21828.0,73,20191202,110439753
1032,9096004,1.0,21828.0,73,20191207,113171287
1375,9096004,1.0,22900.0,33,20191207,113269960
1378,9096004,1.0,22900.0,33,20191206,112507684
1395,9096004,1.0,22900.0,33,20191202,110535131
...,...,...,...,...,...,...
56509,9096004,1.0,23000.0,17727,20191207,113525501
56520,9096004,1.0,23000.0,17727,20191208,113616530
56552,9096004,1.0,23000.0,17727,20191204,111319539
56566,9096004,1.0,23000.0,17727,20191205,111633091


In [16]:
df1a = df1.drop_duplicates()
df1a

,product_ean,sales_pieces,sales_turnover_gross,store_id,sales_date_id,sales_receipt
0,9096004,3.0,65700.0,64,20191201,110342232
1,9096004,2.0,43800.0,64,20191205,111790500
2,9096004,1.0,21900.0,64,20191205,111839706
3,9096004,3.0,65700.0,64,20191204,111004662
4,9096004,1.0,21900.0,64,20191204,110997769
...,...,...,...,...,...,...
56653,9110167,1.0,22700.0,17927,20191207,113469946
56654,9110167,1.0,22700.0,17927,20191208,113616616
56655,9096004,1.0,22500.0,17927,20191208,113958296
56656,9096004,1.0,22500.0,17927,20191204,111373573


Check the dtypes of the dataframe

In [17]:
df1a.dtypes

product_ean               int64
sales_pieces            float64
sales_turnover_gross    float64
store_id                  int64
sales_date_id             int64
sales_receipt             int64
dtype: object

Change the dtype of sales_date_id to a datetime object

In [19]:
df1a.loc[:, 'sales_date_id'] = pd.to_datetime(df1a.loc[:,'sales_date_id'], format = '%Y%m%d').copy()

In [20]:
df1a.dtypes

product_ean                      int64
sales_pieces                   float64
sales_turnover_gross           float64
store_id                         int64
sales_date_id           datetime64[ns]
sales_receipt                    int64
dtype: object

In [22]:
df1a

,product_ean,sales_pieces,sales_turnover_gross,store_id,sales_date_id,sales_receipt
0,9096004,3.0,65700.0,64,2019-12-01,110342232
1,9096004,2.0,43800.0,64,2019-12-05,111790500
2,9096004,1.0,21900.0,64,2019-12-05,111839706
3,9096004,3.0,65700.0,64,2019-12-04,111004662
4,9096004,1.0,21900.0,64,2019-12-04,110997769
...,...,...,...,...,...,...
56653,9110167,1.0,22700.0,17927,2019-12-07,113469946
56654,9110167,1.0,22700.0,17927,2019-12-08,113616616
56655,9096004,1.0,22500.0,17927,2019-12-08,113958296
56656,9096004,1.0,22500.0,17927,2019-12-04,111373573


Check if the duration is correct

In [23]:
print("earliest date recorded is {}, latest date recorded is {}.".format(min(df1a['sales_date_id']),max(df1a['sales_date_id'])))

earliest date recorded is 2019-12-01 00:00:00, latest date recorded is 2019-12-15 00:00:00.


Duration appears to be correct


Now move on to joining the two tables

In [24]:
df2 = pd.DataFrame(pd.read_csv(data['s1_product_dimensions']))

In [25]:
df2

,product_name,is_multipack,product_ean
0,Cigg B,1,9811026
1,Cigg A,0,9096004
2,Cigg A,1,9424999
3,Cigg B,0,9110167


In [26]:
maindf1 = df1a.join(df2.set_index('product_ean'), how = 'inner',on='product_ean')

In [27]:
maindf1

,product_ean,sales_pieces,sales_turnover_gross,store_id,sales_date_id,sales_receipt,product_name,is_multipack
0,9096004,3.0,65700.0,64,2019-12-01,110342232,Cigg A,0
1,9096004,2.0,43800.0,64,2019-12-05,111790500,Cigg A,0
2,9096004,1.0,21900.0,64,2019-12-05,111839706,Cigg A,0
3,9096004,3.0,65700.0,64,2019-12-04,111004662,Cigg A,0
4,9096004,1.0,21900.0,64,2019-12-04,110997769,Cigg A,0
...,...,...,...,...,...,...,...,...
56401,9811026,1.0,216000.0,18113,2019-12-05,111653970,Cigg B,1
56402,9811026,1.0,216000.0,18113,2019-12-05,111606329,Cigg B,1
56403,9811026,1.0,216000.0,18113,2019-12-02,110366546,Cigg B,1
56404,9811026,5.0,1080000.0,18113,2019-12-02,110471522,Cigg B,1


### Investigating the number of sales pieces

In [28]:
print(maindf1['sales_pieces'].unique().tolist())

[3.0, 2.0, 1.0, 6.0, 8.0, 5.0, 4.0, 10.0, 7.0, 50.0, 16.0, 12.0, 30.0, 20.0, 40.0, 11.0, 9.0, 0.2, 15.0, 0.1, 0.25, 0.5, 0.37, 0.15, 14.0, 1000.0, 35.0, 101.0, 100.0, 300.0, 80.0, 200.0, 120.0, 45.0, 43.0, 13.0, 27.0, 29.0, 25.0, 60.0, 63.0, 18.0, 21.0, 86.0, 1.5, 24.0]


### Checking column sales_pieces

When checking the sales pieces column, it appears that there are partial sales pieces which do not make sense. Cigarattes are normally sold by the pack so there must be some issue with the recording of sales. Looking at whether this is regarding multipacks

In [29]:
maindf1.loc[(maindf1['is_multipack'] == 1) & (maindf1['sales_pieces'] < 1)]

,product_ean,sales_pieces,sales_turnover_gross,store_id,sales_date_id,sales_receipt,product_name,is_multipack
10654,9424999,0.5,108250.0,655,2019-12-02,110504197,Cigg A,1
10660,9424999,0.5,108250.0,655,2019-12-02,110488762,Cigg A,1
10670,9424999,0.5,108250.0,655,2019-12-10,114822349,Cigg A,1
10676,9424999,0.5,108250.0,655,2019-12-06,112183239,Cigg A,1
10709,9424999,0.5,108250.0,655,2019-12-12,115368123,Cigg A,1
...,...,...,...,...,...,...,...,...
32504,9811026,0.5,108000.0,17840,2019-12-01,110186101,Cigg B,1
32505,9811026,0.5,108000.0,17840,2019-12-01,110016044,Cigg B,1
32510,9811026,0.5,108000.0,17840,2019-12-04,110984239,Cigg B,1
32512,9811026,0.5,108000.0,17840,2019-12-02,110331961,Cigg B,1


In [30]:
maindf1.loc[(maindf1['is_multipack'] == 1) & (maindf1['sales_pieces'] % maindf1['sales_pieces'].apply(
    lambda x :math.floor(x) if x > 1 else 1) != 0)]

,product_ean,sales_pieces,sales_turnover_gross,store_id,sales_date_id,sales_receipt,product_name,is_multipack
10654,9424999,0.5,108250.0,655,2019-12-02,110504197,Cigg A,1
10660,9424999,0.5,108250.0,655,2019-12-02,110488762,Cigg A,1
10670,9424999,0.5,108250.0,655,2019-12-10,114822349,Cigg A,1
10676,9424999,0.5,108250.0,655,2019-12-06,112183239,Cigg A,1
10701,9424999,1.5,324750.0,655,2019-12-10,114889408,Cigg A,1
...,...,...,...,...,...,...,...,...
32504,9811026,0.5,108000.0,17840,2019-12-01,110186101,Cigg B,1
32505,9811026,0.5,108000.0,17840,2019-12-01,110016044,Cigg B,1
32510,9811026,0.5,108000.0,17840,2019-12-04,110984239,Cigg B,1
32512,9811026,0.5,108000.0,17840,2019-12-02,110331961,Cigg B,1


In [31]:
maindf1.loc[(maindf1['is_multipack'] == 0) & (maindf1['sales_pieces'] % maindf1['sales_pieces'].apply(
            lambda x :math.floor(x) if x > 1 else 1) != 0)]

,product_ean,sales_pieces,sales_turnover_gross,store_id,sales_date_id,sales_receipt,product_name,is_multipack
17182,9096004,0.20,23200.0,16153,2019-12-10,115022502,Cigg A,0
30917,9096004,0.10,3500.0,143,2019-12-12,115607395,Cigg A,0
30918,9096004,0.25,8750.0,143,2019-12-14,116028054,Cigg A,0
30919,9096004,0.25,8750.0,143,2019-12-14,116214674,Cigg A,0
30920,9096004,0.25,8750.0,143,2019-12-13,115901250,Cigg A,0
30921,9096004,0.50,17500.0,143,2019-12-14,116162730,Cigg A,0
30922,9096004,0.37,12950.0,143,2019-12-14,116164219,Cigg A,0
30955,9096004,0.25,7750.0,143,2019-12-01,109979974,Cigg A,0
30956,9096004,0.25,7750.0,143,2019-12-02,110393073,Cigg A,0
30957,9096004,0.25,7750.0,143,2019-12-04,111177452,Cigg A,0


In [32]:
len(maindf1.loc[(maindf1['is_multipack'] == 0) & (maindf1['sales_pieces'] % maindf1['sales_pieces'].apply(
            lambda x :math.floor(x) if x > 1 else 1) != 0)])

19

However we discover that both cases have partial sales_pieces. Multipacks have more cases of partial sales pieces as compared to non-multipacks. Looking at the stores that have partial sales piece

In [33]:
# stores with partial sales
partial_sales_stores = maindf1.loc[(maindf1['sales_pieces'] % maindf1['sales_pieces'].apply(
    lambda x :math.floor(x) if x > 1 else 1) != 0)]['store_id'].unique().tolist()

In [34]:
print(partial_sales_stores)

[16153, 143, 16154, 655, 17840]


Therefore, out of 488 stores, there are 5 stores that have partial sales and together they account for roughly 295 records out of the 56500 records. We next move on to the issue of duplicated sales receipts 

In [35]:
maindf1.loc[:,['sales_receipt','sales_date_id']].drop_duplicates()

,sales_receipt,sales_date_id
0,110342232,2019-12-01
1,111790500,2019-12-05
2,111839706,2019-12-05
3,111004662,2019-12-04
4,110997769,2019-12-04
...,...,...
56399,110778217,2019-12-03
56401,111653970,2019-12-05
56403,110366546,2019-12-02
56404,110471522,2019-12-02


In [36]:
print(len(maindf1), len(maindf1.loc[:,['sales_receipt','sales_date_id']].drop_duplicates()))

56500 53536


Upon close inspection of the pair (sales_receipt and sales_date_id), we find that the number of unique pairs does not equal to the total number of records in our data frame. Hence there is a possibility that the receipt contains two purchases, each for different pack of ciggaratte, to explore this

In [37]:
maindf1.loc[maindf1.duplicated(subset = ['sales_receipt', 'sales_date_id'])]

,product_ean,sales_pieces,sales_turnover_gross,store_id,sales_date_id,sales_receipt,product_name,is_multipack
651,9096004,1.0,18600.0,110,2019-12-01,110354241,Cigg A,0
1367,9096004,2.0,33198.0,33,2019-12-08,113987348,Cigg A,0
1672,9096004,2.0,33200.0,33,2019-12-01,110229431,Cigg A,0
4003,9096004,6.0,130200.0,312,2019-12-06,112766449,Cigg A,0
4082,9096004,1.0,216000.0,312,2019-12-10,114811960,Cigg A,0
...,...,...,...,...,...,...,...,...
56393,9811026,3.0,648000.0,18113,2019-12-02,110445262,Cigg B,1
56397,9811026,1.0,216000.0,18113,2019-12-03,110881625,Cigg B,1
56398,9811026,4.0,864000.0,18113,2019-12-03,110800933,Cigg B,1
56400,9811026,1.0,216000.0,18113,2019-12-03,110878761,Cigg B,1


In [38]:
tempdf1 = maindf1.loc[maindf1.duplicated(subset = ['sales_receipt', 'sales_date_id', 'store_id'])] .groupby(by = ['store_id','sales_receipt','sales_date_id']).count()

In [39]:
tempdf1.loc[tempdf1['sales_pieces']>1]

product_ean  sales_pieces  \
store_id sales_receipt sales_date_id                              
312      112766449     2019-12-06               2             2   
558      110171529     2019-12-01               2             2   
         116381421     2019-12-15               2             2   
931      110218038     2019-12-01               2             2   
         113290545     2019-12-07               2             2   
         113894406     2019-12-08               2             2   
934      110419265     2019-12-02               2             2   
15454    114654213     2019-12-09               2             2   
15947    115384250     2019-12-12               2             2   
16764    118141875     2019-12-10               2             2   
17103    111308892     2019-12-04               3             3   
         115325633     2019-12-11               2             2   
17188    113949289     2019-12-08               2             2   
         116179523     2019-12-14               2             2   
         116332542     2019-12-14               2             2   
         116496924     2019-12-15               2             2   
         116573866     2019-12-15               2             2   
17840    113516380     2019-12-07               2             2   

                                      sales_turnover_gross  product_name  \
store_id sales_receipt sales_date_id                                       
312      112766449     2019-12-06                        2             2   
558      110171529     2019-12-01                        2             2   
         116381421     2019-12-15                        2             2   
931      110218038     2019-12-01                        2             2   
         113290545     2019-12-07                        2             2   
         113894406     2019-12-08                        2             2   
934      110419265     2019-12-02                        2             2   
15454    114654213     2019-12-09                        2             2   
15947    115384250     2019-12-12                        2             2   
16764    118141875     2019-12-10                        2             2   
17103    111308892     2019-12-04                        3             3   
         115325633     2019-12-11                        2             2   
17188    113949289     2019-12-08                        2             2   
         116179523     2019-12-14                        2             2   
         116332542     2019-12-14                        2             2   
         116496924     2019-12-15                        2             2   
         116573866     2019-12-15                        2             2   
17840    113516380     2019-12-07                        2             2   

                                      is_multipack  
store_id sales_receipt sales_date_id                
312      112766449     2019-12-06                2  
558      110171529     2019-12-01                2  
         116381421     2019-12-15                2  
931      110218038     2019-12-01                2  
         113290545     2019-12-07                2  
         113894406     2019-12-08                2  
934      110419265     2019-12-02                2  
15454    114654213     2019-12-09                2  
15947    115384250     2019-12-12                2  
16764    118141875     2019-12-10                2  
17103    111308892     2019-12-04                3  
         115325633     2019-12-11                2  
17188    113949289     2019-12-08                2  
         116179523     2019-12-14                2  
         116332542     2019-12-14                2  
         116496924     2019-12-15                2  
         116573866     2019-12-15                2  
17840    113516380     2019-12-07                2

Hence we can see that there are some shops which have entered the same receipt but as two seperate purchases or even 3 seperate purchases, the stores are selling more than 1 type of ciggarette to the same customer as the sales receipt is the same, looking closely at this store ids 

In [40]:
maindf1.loc[(maindf1['store_id'] == 312) & (maindf1['sales_receipt'] ==  112766449)]

,product_ean,sales_pieces,sales_turnover_gross,store_id,sales_date_id,sales_receipt,product_name,is_multipack
3987,9096004,1.0,216000.0,312,2019-12-06,112766449,Cigg A,0
4003,9096004,6.0,130200.0,312,2019-12-06,112766449,Cigg A,0
4112,9811026,3.0,65700.0,312,2019-12-06,112766449,Cigg B,1


We observe here that for store_id 312, the store has three records with the same sales_receipt number and on the same day as well. We also observe that the purchase of the same type of ciggarette 'Cigg A' which is not a multipack is recorded twice. Further we observe that the price of a single pack of Cigg A is different across the two records, this might be due to the fact that a higher brand of cigg is sold in the first record. We need to introduce another column which would indicate that there is a duplicate receipt, so that issues such as this can be identified in advance

In [41]:
maindf1['sales_receipt_has_duplicate'] = maindf1.duplicated(subset = 'sales_receipt', keep = False) # set bool column

In [42]:
maindf1.loc[(maindf1['store_id'] == 312) & (maindf1['sales_receipt'] ==  112766449)]

,product_ean,sales_pieces,sales_turnover_gross,store_id,sales_date_id,sales_receipt,product_name,is_multipack,sales_receipt_has_duplicate
3987,9096004,1.0,216000.0,312,2019-12-06,112766449,Cigg A,0,True
4003,9096004,6.0,130200.0,312,2019-12-06,112766449,Cigg A,0,True
4112,9811026,3.0,65700.0,312,2019-12-06,112766449,Cigg B,1,True


In [43]:
maindf1.loc[(maindf1['is_multipack'] == 1) & (maindf1['sales_pieces'] % maindf1['sales_pieces'].apply(
            lambda x :math.floor(x) if x > 1 else 1) != 0) & (maindf1['sales_receipt_has_duplicate'] == True)]

,product_ean,sales_pieces,sales_turnover_gross,store_id,sales_date_id,sales_receipt,product_name,is_multipack,sales_receipt_has_duplicate
29842,9424999,0.5,106750.0,17840,2019-12-14,116056203,Cigg A,1,True
30128,9424999,0.5,106750.0,17840,2019-12-14,116112000,Cigg A,1,True
30139,9424999,0.5,106750.0,17840,2019-12-15,116540136,Cigg A,1,True
30255,9424999,0.5,106750.0,17840,2019-12-13,115755728,Cigg A,1,True
30273,9424999,0.5,106750.0,17840,2019-12-11,115070073,Cigg A,1,True
...,...,...,...,...,...,...,...,...,...
32484,9811026,0.5,108000.0,17840,2019-12-02,110390413,Cigg B,1,True
32493,9811026,0.5,108000.0,17840,2019-12-02,110324310,Cigg B,1,True
32497,9811026,0.5,108000.0,17840,2019-12-02,110287549,Cigg B,1,True
32503,9811026,0.5,108000.0,17840,2019-12-02,110581888,Cigg B,1,True


In [44]:
maindf1.loc[(maindf1['is_multipack'] == 1) & (maindf1['sales_pieces'] % maindf1['sales_pieces'].apply(
            lambda x :math.floor(x) if x > 1 else 1) != 0)] # partial sales for multipack 

,product_ean,sales_pieces,sales_turnover_gross,store_id,sales_date_id,sales_receipt,product_name,is_multipack,sales_receipt_has_duplicate
10654,9424999,0.5,108250.0,655,2019-12-02,110504197,Cigg A,1,False
10660,9424999,0.5,108250.0,655,2019-12-02,110488762,Cigg A,1,False
10670,9424999,0.5,108250.0,655,2019-12-10,114822349,Cigg A,1,False
10676,9424999,0.5,108250.0,655,2019-12-06,112183239,Cigg A,1,False
10701,9424999,1.5,324750.0,655,2019-12-10,114889408,Cigg A,1,False
...,...,...,...,...,...,...,...,...,...
32504,9811026,0.5,108000.0,17840,2019-12-01,110186101,Cigg B,1,True
32505,9811026,0.5,108000.0,17840,2019-12-01,110016044,Cigg B,1,False
32510,9811026,0.5,108000.0,17840,2019-12-04,110984239,Cigg B,1,False
32512,9811026,0.5,108000.0,17840,2019-12-02,110331961,Cigg B,1,False


In [85]:
# clean cigg_names
maindf1.loc[:,'product_name'] = maindf1['product_name'].apply(lambda x: x.strip())

### In order to deal with the above issues, suggest a function that can be applied to future data frames that can manage this issues

In [153]:
def solution1(path = os.getcwd(), remove_duplicates = True, select_data = True, check_period = True,
              check_partial_sales = True):
    """
    entire process of reading in data frame from file directory,
    pre-processing, and data mutation
    path : path where data is located
    remove_duplicates : if True then only first entry for duplicate rows (entire row) will remain
    select_data : if path has more than the two desired csv files, transaction data and product dimensions,
    user will be prompted to select the data
    check_period : if true, will print start and end date of the data
    check_partial_sales : if true, will print out store ids with partial sales if any, partial sales refer to floating
    point numbers
    """
    data = {x.split('.csv')[0]:x for x in os.listdir() if '.csv' in x}
    names = list(data.keys())
    if select_data:
        for i in range(len(names)):
            print(names[i], ' : ' ,i + 1)
        print()
        transaction_data = input('Please key in the transaction file number from the list above: ')
        transaction_data = names[int(transaction_data)-1]
        for i in range(len(names)):
            if names[i] == transaction_data:
                continue
            else:
                print(names[i], ' : ',i + 1)
        print()
        dimension_data = input('Please key in the dimension file number from the above list: ' )
        dimension_data = names[int(dimension_data)-1]
    else:    
        if 'transaction' in names[0]:
            transaction_data = names[0]
            dimension_data = names[1]
        else:
            transaction_data = names[1]
            dimension_data = names[0]
    
    def join_data(name1, name2, remove_duplicates, check_period, check_partial_sales):
        """
        name1 : transaction data
        name2 : dimension data
        """
        print("Creating data frame\n")
        df1 = pd.DataFrame(pd.read_csv(data[name1]))
        df2 = pd.DataFrame(pd.read_csv(data[name2]))
        if remove_duplicates:
            df1 = df1.drop_duplicates().copy()        
        df1.loc[:, 'sales_date_id'] = pd.to_datetime(df1.loc[:,'sales_date_id'], format = '%Y%m%d').copy()
        if check_period:
            print("Earliest date recorded is {}, latest date recorded is {}.".format(min(df1['sales_date_id']),max(df1['sales_date_id'])))
            print()
        maindf1 = df1.join(df2.set_index('product_ean'), how = 'inner',on='product_ean')
        maindf1['sales_receipt_has_duplicate'] = maindf1.duplicated(subset = 'sales_receipt', keep = False) # set bool column
        if check_partial_sales:
            partial_sales_stores = maindf1.loc[(maindf1['sales_pieces'] % maindf1['sales_pieces'].apply(
            lambda x :math.floor(x) if x > 1 else 1) != 0)]['store_id'].unique().tolist()
            print("List of stores with partial sales: ", partial_sales_stores)
            print()
        maindf1.loc[:,'product_name'] = maindf1['product_name'].apply(lambda x: x.strip())
        temp_index_ls = maindf1.loc[(maindf1['is_multipack'] == 0) & (maindf1['sales_pieces'] % maindf1['sales_pieces'].apply(
            lambda x :math.floor(x) if x > 1 else 1) != 0)].index.to_list()
        maindf1 = maindf1.loc[~maindf1.index.isin(temp_index_ls)].copy() # remove partial sales from non multipack cigarettes

        print("=" * len("Data Frame Created"))
        print("Data Frame Created")
        print("=" * len("Data Frame Created"))
        return maindf1

    return join_data(transaction_data, dimension_data, remove_duplicates, check_period, check_partial_sales)
    
    
    
    


In [154]:
maindf2 = solution1(select_data=False)

Creating data frame

Earliest date recorded is 2019-12-01 00:00:00, latest date recorded is 2019-12-15 00:00:00.

List of stores with partial sales:  [16153, 143, 16154, 655, 17840]

Data Frame Created


Checking the output data frame

### Requirement 
2. Estimate the total number of units sold across the time period for each cigarette. Applications are examined based on the quality of considerations that was given to the estimation process

In [156]:
maindf2.loc[(maindf2['is_multipack'] == 1) & (maindf2['sales_pieces'] % maindf2['sales_pieces'].apply(
            lambda x :math.floor(x) if x > 1 else 1) != 0)]

,product_ean,sales_pieces,sales_turnover_gross,store_id,sales_date_id,sales_receipt,product_name,is_multipack,sales_receipt_has_duplicate
10654,9424999,0.5,108250.0,655,2019-12-02,110504197,Cigg A,1,False
10660,9424999,0.5,108250.0,655,2019-12-02,110488762,Cigg A,1,False
10670,9424999,0.5,108250.0,655,2019-12-10,114822349,Cigg A,1,False
10676,9424999,0.5,108250.0,655,2019-12-06,112183239,Cigg A,1,False
10701,9424999,1.5,324750.0,655,2019-12-10,114889408,Cigg A,1,False
...,...,...,...,...,...,...,...,...,...
32504,9811026,0.5,108000.0,17840,2019-12-01,110186101,Cigg B,1,True
32505,9811026,0.5,108000.0,17840,2019-12-01,110016044,Cigg B,1,False
32510,9811026,0.5,108000.0,17840,2019-12-04,110984239,Cigg B,1,False
32512,9811026,0.5,108000.0,17840,2019-12-02,110331961,Cigg B,1,False


In [60]:
temp_index_ls = maindf2.loc[(maindf2['is_multipack'] == 0) & (maindf2['sales_pieces'] % maindf2['sales_pieces'].apply(
            lambda x :math.floor(x) if x > 1 else 1) != 0)].index.to_list()

Partial sales for non multipack leads to partial total of sales peices

In [150]:
maindf2.loc[:,('sales_pieces', 'product_name','is_multipack')].groupby(
    by = ['product_name', 'is_multipack']).sum()

sales_pieces
product_name is_multipack              
Cigg A       0                 56717.72
             1                 16738.50
Cigg B       0                 27010.00
             1                  8075.50

In [112]:
maindf2.loc[~maindf2.index.isin(temp_index_ls),('sales_pieces', 'product_name','is_multipack')].groupby(
    by = ['product_name', 'is_multipack']).sum()

sales_pieces
product_name is_multipack              
Cigg A       0                  56713.0
             1                  16738.5
Cigg B       0                  27010.0
             1                   8075.5

#### Methods

- Given all other features except for the sales_peices, we can estimate the number of units of each ciggarate based on the per unit price of each cigg.
- After estimating the per unit price of each ciggarate

In [157]:
tempdf1 = maindf2.loc[:,['sales_pieces', 'store_id', 'sales_date_id']].groupby(by = ['sales_date_id','store_id']).sum().sort_index(by = 'sales_date_id', ascending = True).copy()

C:\Users\RICHARD IRVING\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: FutureWarning: by argument to sort_index is deprecated, please use .sort_values(by=...)
  """Entry point for launching an IPython kernel.


Check for any price volatility 

In [160]:
maindf2

,product_ean,sales_pieces,sales_turnover_gross,store_id,sales_date_id,sales_receipt,product_name,is_multipack,sales_receipt_has_duplicate
0,9096004,3.0,65700.0,64,2019-12-01,110342232,Cigg A,0,False
1,9096004,2.0,43800.0,64,2019-12-05,111790500,Cigg A,0,False
2,9096004,1.0,21900.0,64,2019-12-05,111839706,Cigg A,0,False
3,9096004,3.0,65700.0,64,2019-12-04,111004662,Cigg A,0,False
4,9096004,1.0,21900.0,64,2019-12-04,110997769,Cigg A,0,False
...,...,...,...,...,...,...,...,...,...
56401,9811026,1.0,216000.0,18113,2019-12-05,111653970,Cigg B,1,False
56402,9811026,1.0,216000.0,18113,2019-12-05,111606329,Cigg B,1,True
56403,9811026,1.0,216000.0,18113,2019-12-02,110366546,Cigg B,1,False
56404,9811026,5.0,1080000.0,18113,2019-12-02,110471522,Cigg B,1,False


In [177]:
tempdf1 = maindf2.copy()

In [178]:
tempdf1['price_per_unit'] = tempdf1['sales_turnover_gross']/tempdf1['sales_pieces']

In [179]:
tempdf1

,product_ean,sales_pieces,sales_turnover_gross,store_id,sales_date_id,sales_receipt,product_name,is_multipack,sales_receipt_has_duplicate,price_per_unit
0,9096004,3.0,65700.0,64,2019-12-01,110342232,Cigg A,0,False,21900.0
1,9096004,2.0,43800.0,64,2019-12-05,111790500,Cigg A,0,False,21900.0
2,9096004,1.0,21900.0,64,2019-12-05,111839706,Cigg A,0,False,21900.0
3,9096004,3.0,65700.0,64,2019-12-04,111004662,Cigg A,0,False,21900.0
4,9096004,1.0,21900.0,64,2019-12-04,110997769,Cigg A,0,False,21900.0
...,...,...,...,...,...,...,...,...,...,...
56401,9811026,1.0,216000.0,18113,2019-12-05,111653970,Cigg B,1,False,216000.0
56402,9811026,1.0,216000.0,18113,2019-12-05,111606329,Cigg B,1,True,216000.0
56403,9811026,1.0,216000.0,18113,2019-12-02,110366546,Cigg B,1,False,216000.0
56404,9811026,5.0,1080000.0,18113,2019-12-02,110471522,Cigg B,1,False,216000.0


In [176]:
maindf2.groupby('store_id').groups[4]

Int64Index([ 264,  265,  266,  267,  268,  269,  270,  271,  272,  273,
            ...
            4671, 4672, 4673, 4674, 4675, 4676, 4677, 4678, 4679, 4680],
           dtype='int64', length=457)

In [181]:
def volatile(series):
    """
    series is the per unit price
    """
    state = False
    if len(series.unique()) != 1:
        state = True
    return state

In [197]:
tempdf2 = tempdf1.groupby('store_id').agg({'price_per_unit': volatile}).copy()

In [201]:
tempdf2 = tempdf1.groupby(['store_id','is_multipack']).agg({'price_per_unit': volatile}).copy()

In [202]:
tempdf2

price_per_unit
store_id is_multipack                
4        0                       True
         1                       True
20       0                       True
21       0                      False
23       0                      False
...                               ...
18761    0                       True
18765    0                       True
18772    0                      False
18773    0                       True
18791    0                       True

[591 rows x 1 columns]

In [243]:
tempdf3 = tempdf2.loc[tempdf2['price_per_unit'] == True].copy() # cannot index on view, will return original index

In [244]:
tempdf3

price_per_unit
store_id is_multipack                
4        0                       True
         1                       True
20       0                       True
24       0                       True
26       0                       True
...                               ...
18730    0                       True
18761    0                       True
18765    0                       True
18773    0                       True
18791    0                       True

[410 rows x 1 columns]

In [249]:
len(tempdf3.droplevel('is_multipack').index.unique())

359

Out of **488** stores, **359** stores have prices that change from one record to another, hence they majority do not have steady prices

In [252]:
tempdf1.groupby(('product_name','is_multipack')).sum()

C:\Users\RICHARD IRVING\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: FutureWarning: Interpreting tuple 'by' as a list of keys, rather than a single key. Use 'by=[...]' instead of 'by=(...)'. In the future, a tuple will always mean a single key.
  """Entry point for launching an IPython kernel.


product_ean  sales_pieces  sales_turnover_gross  \
product_name is_multipack                                                     
Cigg A       0             278392298424       56713.0          1.858077e+09   
             1              84127541074       16738.5          9.145293e+08   
Cigg B       0             128207380191       27010.0          7.363878e+08   
             1              28216510776        8075.5          5.663033e+08   

                            store_id  sales_receipt  \
product_name is_multipack                             
Cigg A       0             284412064  3478230232277   
             1              92265517  1013640701900   
Cigg B       0             141451844  1599865537675   
             1              31743205   326601354940   

                           sales_receipt_has_duplicate  price_per_unit  
product_name is_multipack                                               
Cigg A       0                                  1977.0    1.169031e+09  
             1                                  1005.0    6.044213e+08  
Cigg B       0                                  2087.0    4.472432e+08  
             1                                   840.0    3.130263e+08

#### Methodology

- In order to estimate the number of units sold for each ciggerate type. We will first find the sample mean price per unit of ciggarete for each type of cigg. We store this under the series mean price per unit.
- We will then weight the sample mean by the std dev(price) of each store, the assumption is that the larger the std dev, the higher the penalty  
- the penalty function applied is the sample mean * [(sample mean - std)/(sample mean)]/(total number of shops selling that cigg)
- we then find the sum of the above function

##### Assumptions
- Assume that a multipack contains 10 packs of cigg

In [254]:
tempdf1.loc[tempdf1['is_multipack'] == 1]

,product_ean,sales_pieces,sales_turnover_gross,store_id,sales_date_id,sales_receipt,product_name,is_multipack,sales_receipt_has_duplicate,price_per_unit
252,9424999,1.0,214500.0,287,2019-12-15,116461471,Cigg A,1,False,214500.0
254,9424999,1.0,214500.0,287,2019-12-13,115774012,Cigg A,1,False,214500.0
256,9424999,1.0,214500.0,287,2019-12-14,116060980,Cigg A,1,False,214500.0
257,9424999,1.0,214500.0,287,2019-12-15,116410584,Cigg A,1,False,214500.0
258,9424999,2.0,429000.0,287,2019-12-15,116518912,Cigg A,1,True,214500.0
...,...,...,...,...,...,...,...,...,...,...
56401,9811026,1.0,216000.0,18113,2019-12-05,111653970,Cigg B,1,False,216000.0
56402,9811026,1.0,216000.0,18113,2019-12-05,111606329,Cigg B,1,True,216000.0
56403,9811026,1.0,216000.0,18113,2019-12-02,110366546,Cigg B,1,False,216000.0
56404,9811026,5.0,1080000.0,18113,2019-12-02,110471522,Cigg B,1,False,216000.0


In [257]:
tempdf1.loc[:,('sales_pieces','is_multipack')]

,sales_pieces,is_multipack
0,3.0,0
1,2.0,0
2,1.0,0
3,3.0,0
4,1.0,0
...,...,...
56401,1.0,1
56402,1.0,1
56403,1.0,1
56404,5.0,1


In [259]:
def total_pieces(series):
    return series[0]*10 if series[1] == 1 else series[0]

In [377]:
# column for total pieces

tempdf1['total_pieces'] = tempdf1.loc[:,('sales_pieces','is_multipack')].apply(
    total_pieces, axis = 1)

In [260]:
tempdf1['price_per_unit'] = tempdf1['sales_turnover_gross']/(tempdf1.loc[:,('sales_pieces','is_multipack')].apply(
    total_pieces, axis = 1))

In [284]:
grouped1 = tempdf1.loc[tempdf1['product_name'] == 'Cigg A', ('store_id', 'price_per_unit')].groupby('store_id')

In [288]:
grouped2 = tempdf1.loc[tempdf1['product_name'] == 'Cigg B', ('store_id', 'price_per_unit')].groupby('store_id')

In [380]:
tempdf1[(tempdf1['store_id']==934) & (tempdf1['is_multipack'] == 1)]

,product_ean,sales_pieces,sales_turnover_gross,store_id,sales_date_id,sales_receipt,product_name,is_multipack,sales_receipt_has_duplicate,price_per_unit,total_pieces
14425,9811026,1.0,22000.0,934,2019-12-11,115143253,Cigg B,1,False,2200.0,10.0
14426,9811026,2.0,44000.0,934,2019-12-12,115501408,Cigg B,1,False,2200.0,20.0
14427,9811026,5.0,110000.0,934,2019-12-12,115522680,Cigg B,1,True,2200.0,50.0
14428,9811026,5.0,110000.0,934,2019-12-13,115827716,Cigg B,1,False,2200.0,50.0
14429,9811026,1.0,216500.0,934,2019-12-13,115844897,Cigg B,1,False,21650.0,10.0
14430,9811026,1.0,22000.0,934,2019-12-13,115794438,Cigg B,1,False,2200.0,10.0
14431,9811026,1.0,22000.0,934,2019-12-14,116269110,Cigg B,1,False,2200.0,10.0
15400,9811026,1.0,216500.0,934,2019-12-13,115963563,Cigg B,1,False,21650.0,10.0
15401,9811026,1.0,22000.0,934,2019-12-11,115244665,Cigg B,1,False,2200.0,10.0
15402,9811026,1.0,22000.0,934,2019-12-10,115023927,Cigg B,1,False,2200.0,10.0


In [336]:
grouped1.agg(['std', 'mean']).sort_values(by = ('price_per_unit','mean')).iloc[934]

price_per_unit               
                    std           mean
store_id                              
15723        540.117509    1984.615385
52             0.000000    2145.000000
853            4.604072    2147.588235
17312         21.532889    2149.806818
36           125.335149    2152.274590
...                 ...            ...
934       169241.196042   81103.482587
15386      92522.011437   91214.114833
558       138895.568689  148392.409532
1026      282966.598741  152074.408848
1025      271980.229066  176768.032787

[473 rows x 2 columns]

In [348]:
# aggregation functions

def after_penalty(series):
    score = series[1]
    if series[0] > 0: # apply the penalty
        if series[0] > series[1]: # std more than mean
            score = 0 # exclude from the series
        else:
            score = series[1] * ((series[1]-series[0])/series[1])
    return score


def summation(ls1):
    total = 0
    counter = 0
    for price in ls1:
        if price == 0:
            continue
        else:
            total += price
            counter += 1
    return total/counter
    

In [346]:
ciggA_list = list(grouped1.agg(['std', 'mean']).sort_values(by = ('price_per_unit','mean')).apply(after_penalty, axis = 1).values)

In [349]:
ciggA_price = summation(ciggA_list)

In [350]:
ciggA_price

19436.616173057497

In [290]:
grouped1.filter(lambda x : x['price_per_unit'].std() > 0)`

,store_id,price_per_unit
70,81,21900.0
71,81,21900.0
72,81,21700.0
73,81,21900.0
74,81,21900.0
...,...,...
56457,18113,21400.0
56458,18113,21400.0
56459,18113,21400.0
56460,18113,21400.0


In [291]:
grouped2.filter(lambda x : x['price_per_unit'].std() > 0)

,store_id,price_per_unit
82,81,22100.0
83,81,21900.0
84,81,22100.0
85,81,22100.0
86,81,22100.0
...,...,...
56401,18113,21600.0
56402,18113,21600.0
56403,18113,21600.0
56404,18113,21600.0


In [282]:
tempdf1[tempdf1['store_id'] == 64]

,product_ean,sales_pieces,sales_turnover_gross,store_id,sales_date_id,sales_receipt,product_name,is_multipack,sales_receipt_has_duplicate,price_per_unit
0,9096004,3.0,65700.0,64,2019-12-01,110342232,Cigg A,0,False,21900.0
1,9096004,2.0,43800.0,64,2019-12-05,111790500,Cigg A,0,False,21900.0
2,9096004,1.0,21900.0,64,2019-12-05,111839706,Cigg A,0,False,21900.0
3,9096004,3.0,65700.0,64,2019-12-04,111004662,Cigg A,0,False,21900.0
4,9096004,1.0,21900.0,64,2019-12-04,110997769,Cigg A,0,False,21900.0
...,...,...,...,...,...,...,...,...,...,...
65,9110167,1.0,22000.0,64,2019-12-15,116378725,Cigg B,0,False,22000.0
66,9110167,1.0,22000.0,64,2019-12-15,116514277,Cigg B,0,False,22000.0
67,9110167,1.0,22000.0,64,2019-12-15,116491195,Cigg B,0,False,22000.0
68,9110167,1.0,22000.0,64,2019-12-14,116136509,Cigg B,0,False,22000.0


In [294]:
len(grouped.filter(lambda x : x['price_per_unit'].std() > 0)['store_id'].unique()) # this is wrong, must differentiate 
# between Cigg A and Cigg B

383

In [292]:
len(grouped1.filter(lambda x : x['price_per_unit'].std() > 0)['store_id'].unique()) 

202

In [293]:
len(grouped2.filter(lambda x : x['price_per_unit'].std() > 0)['store_id'].unique()) 

159

Out of 488 stores, 361 have prices which are volatile, the prices have a std > 0.

In [434]:
def estimate(dataframe = tempdf1):
    """
    returns a dict of prices
    """
    
    ciggs = ('Cigg A', 'Cigg B')
    results = {}
    
    def after_penalty(series):
        score = series[1]
        if series[0] > 0: # apply the penalty
            if series[0] > series[1]: # std more than mean
                score = 0 # exclude from the series
            else:
                score = series[1] * ((series[1]-series[0])/series[1])
        return score


    def summation(ls1):
        total = 0
        counter = 0
        for price in ls1:
            if price == 0:
                continue
            else:
                total += price
                counter += 1
        return total/counter
    
    
    for cigg in ciggs:
        grouped1 = tempdf1.loc[tempdf1['product_name'] == cigg, ('store_id', 'price_per_unit')].groupby('store_id')
        grouped1.agg(['std', 'mean']).sort_values(by = ('price_per_unit','mean'))
        cigg_list = list(grouped1.agg(['std', 'mean']).sort_values(by = ('price_per_unit','mean')).apply(
            after_penalty, axis = 1).values)
        results[cigg] = summation(cigg_list)
    
    return results

Get the estimated price per unit for each cigg

In [435]:
prices = estimate(tempdf1)

In [436]:
prices

{'Cigg A': 19436.616173057497, 'Cigg B': 21677.97175821097}

Finally to get the total number of sales unit, we take the gross sales for each cigg and divide it by the estimated price of each cigg.


In [357]:
tempdf1.loc[tempdf1['product_name'] == 'Cigg A', 'sales_turnover_gross'].sum()

2772606173.0

In [437]:
def estimate_sales(dataframe = tempdf1, prices = prices):
    sales = {}
#     total = 0
    for k,v in prices.items():    
        tempdf = tempdf1.loc[tempdf1['product_name'] == k, 'sales_turnover_gross']
#         print(tempdf.shape)
        total_sales = tempdf.sum()
#         total += total_sales
        
        sales[k] = round(total_sales/v,0)
#     print(total)
    return sales
        
        

In [438]:
tempdf1['sales_turnover_gross'].sum()

4075297269.0

In [439]:
sales = estimate_sales(tempdf1, prices)

In [440]:
sales

{'Cigg A': 142649.0, 'Cigg B': 60093.0}

In [441]:
product_name = ['Cigg A', 'Cigg B']
total_pieces = [224098.0,107765.0]
estimated_total_pieces = [142649.0,60093.0]

summarydf = pd.DataFrame(zip(product_name, total_pieces,estimated_total_pieces), columns = ['product_name',
                                                                                           'total_pieces',
                                                                                           'estimated_total_pieces'])

In [442]:
summarydf.set_index('product_name')

,total_pieces,estimated_total_pieces
product_name,,
Cigg A,224098.0,142649.0
Cigg B,107765.0,60093.0


In [443]:
summarydf['error'] = summarydf.apply(lambda x: abs(x[1] - x[2]), axis = 1)

In [444]:
summarydf

,product_name,total_pieces,estimated_total_pieces,error
0,Cigg A,224098.0,142649.0,81449.0
1,Cigg B,107765.0,60093.0,47672.0
